In [1]:
class LRUCache:
    
    def __init__(self, size=32):
        self.size = size
        self.dll = DLL()
        
    def add(self, value):
        value_index = self.dll.get_index_of_value(value)
        if value_index:  # if value is already there we don't have to check for length as we just move it
            self.dll.delete(value_index)
        else:
            if self.dll.length >= self.size:
                self.dll.delete(self.dll.length-1)  # delete one item from tail
        self.dll.prepend(value)  # add to front
    
    def show(self):
        return self.dll.show()

In [2]:
class DLL:
    
    class Node:
        
        def __init__(self, value=None, prev=None, next=None):
            self.value = value
            self.prev = prev
            self.next = next
            
        @classmethod
        def connect(cls, node_prev, node_next):
            if not node_prev and not node_next:
                return
            if not node_prev:
                node_next.prev = None
            if not node_next:
                node_prev.next = None
            if node_prev and node_next:
                node_prev.next = node_next
                node_next.prev = node_prev

            
    def __init__(self):
        self.head = None
        self.tail = None
        
    @property
    def length(self):
        i = 0
        for _ in self._traverse_from_head():
            i += 1
        return i
            
    def append(self, value):
        # add to tail
        if not self.head:
            self.head = self.tail = self.Node(value)
        else:
            node = self.Node(value)
            self.Node().connect(self.tail, node)
            self.tail = node
            
    def prepend(self, value):
        # add before head
        if not self.head:
            self.head = self.tail = self.Node(value)
        else:
            node = self.Node(value)
            self.Node().connect(node, self.head)
            self.head = node
            
    def insert(self, index, value):
        node_new = self.Node(value)
        for i, node in enumerate(self._traverse_from_head()):
            if i == index:
                self.Node().connect(node.prev, node_new)
                self.Node().connect(node_new, node)
                return True
        return None
        
    def get_value_at_index(self, index):
        for i, node in enumerate(self._traverse_from_head()):
            if i == index:
                return node.value
        return None
    
    def get_index_of_value(self, value):
        for i, node in enumerate(self._traverse_from_head()):
            if node.value == value:
                return i
        return None
    
    def delete(self, index):
        # removes node and connects prev and next
        for i, node in enumerate(self._traverse_from_head()):
            if i == index:
                self.Node().connect(node.prev, node.next)
                if not node.prev:
                    self.head = node.next
                if not node.next:
                    self.tail = node.prev
                return True
        return False
    
    def _traverse_from_head(self):
        if not self.head:
            return None
        curr_node = self.head
        while curr_node.next:
            yield curr_node
            curr_node = curr_node.next
        yield curr_node
    
    def _traverse_from_tail(self):
        if not self.tail:
            return None
        curr_node = self.tail
        while curr_node.prev:
            yield curr_node
            curr_node = curr_node.prev
        yield curr_node
    
    def show(self, from_tail=False):
        result = ''
        traverse = self._traverse_from_head()
        if from_tail:
            traverse = self._traverse_from_tail()
        for node in traverse:
            result += '{}, '.format(node.value)
        return result.strip(', ')

In [3]:
import unittest


class TestLRUCache(unittest.TestCase):
    
    def __init__(self, *args, **kwargs):
        super(TestLRUCache, self).__init__(*args, **kwargs)
        self.lruc = LRUCache(4)
        
    def test_lru_cache(self):
        self.lruc.add(1)
        self.lruc.add(2)
        self.lruc.add(3)
        self.lruc.add(2)
        self.lruc.add(5)
        self.lruc.add(3)
        self.lruc.add(4)
        self.lruc.add(5)
        self.lruc.add(8)
        self.lruc.add(9)
        self.assertEqual(self.lruc.show(), '9, 8, 5, 4')
        

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
import unittest


class TestDLL(unittest.TestCase):
    
    def __init__(self, *args, **kwargs):
        super(TestDLL, self).__init__(*args, **kwargs)
        self.dll = DLL()
    
    def test_append(self):
        self.assertEqual(self.dll.show(), '')
        self.assertEqual(self.dll.show(from_tail=True), '')
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.assertEqual(self.dll.show(), '3, 11, rrr')
        self.assertEqual(self.dll.show(from_tail=True), 'rrr, 11, 3')
        
    def test_prepend(self):
        self.assertEqual(self.dll.show(), '')
        self.assertEqual(self.dll.show(from_tail=True), '')
        self.dll.prepend(3)
        self.dll.prepend(11)
        self.dll.prepend('rrr')
        self.assertEqual(self.dll.show(from_tail=True), '3, 11, rrr')
        self.assertEqual(self.dll.show(), 'rrr, 11, 3')
        
    def test_insert(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.assertEqual(self.dll.show(), '3, 11, rrr')
        self.dll.insert(1, 'pp')
        self.assertEqual(self.dll.get_value_at_index(1), 'pp')
        self.assertEqual(self.dll.show(), '3, pp, 11, rrr')
        
    def test_get_value_at_index(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.assertEqual(self.dll.get_value_at_index(0), 3)
        self.assertEqual(self.dll.get_value_at_index(1), 11)
        self.assertEqual(self.dll.get_value_at_index(2), 'rrr')
        
    def test_get_index_of_value(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.assertEqual(self.dll.get_index_of_value(3), 0)
        self.assertEqual(self.dll.get_index_of_value(11), 1)
        self.assertEqual(self.dll.get_index_of_value('rrr'), 2)
        
    def test_delete_middle(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.dll.delete(1)
        self.assertEqual(self.dll.show(), '3, rrr')
        
    def test_delete_head(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.dll.delete(0)
        self.assertEqual(self.dll.show(), '11, rrr')
        
    def test_delete_tail(self):
        self.dll.append(3)
        self.dll.append(11)
        self.dll.append('rrr')
        self.dll.delete(2)
        self.assertEqual(self.dll.show(), '3, 11')
        
    def test_length(self):
        self.assertEqual(self.dll.length, 0)
        self.dll.append(3)
        self.dll.append('rrr')
        self.assertEqual(self.dll.length, 2)
        self.dll.delete(1)
        self.assertEqual(self.dll.length, 1)
        self.dll.delete(0)
        self.assertEqual(self.dll.length, 0)
        

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.006s

OK
